# 浪子之心科技
## 此代码仅为演示循环驱动同屏双人数字人，外加后期超分处理的框架。代码是用wav2lip循环驱动同屏双人数字人，训练高清模型，做流式处理，加强硬件，并且不做后期超分，即可用于双人数字人实时直播。也可以再此框架的基础上，将wav2lip替换成其他的数字人驱动项目。其中视频1-6为提前录制的双人不说话视频，音频1-6为提前准备的说话音频。
### 一、提前将录制好的双人视频素材和音频分别上传temp_video和input_audio。
### 二、此双人数字人素材录制非常关键，音频和视频切割也很关键。
### 三、视频处理速度和GPU性能直接相关，如果需要直播的话，需要进行流式处理，并且使用高性能GPU。
### 四、CodeFormer超分速度较慢，对数字人视频要求不高的可以不做超分，或者换成GFPGAN进行超分。
### 五、代码中没有使用变量，你需要自己写变量。



In [1]:
#@title 一、下载Double_Talker
#@title
!rm -rf /content/sample_data
%cd /content/
!git clone https://github.com/langzizhixin/Double_Talker
%cd /content/Double_Talker
# 下载Wav2Lip，并安装环境
!rm -rf Wav2Lip
!git clone https://github.com/zabique/Wav2Lip
%cd /content/Double_Talker/Wav2Lip
#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Double_Talker/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt
#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Double_Talker/Wav2Lip/face_detection/detection/sfd/s3fd.pth"
!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1
!pip install basicsr
!pip install gradio
#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from IPython.display import clear_output
clear_output()
print("\nDone")


Done


In [2]:
#@title 二、下载CodeFormer的源码权重并安装依赖
# 下载CodeFormer的源码
%cd /content/Double_Talker
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/Double_Talker/CodeFormer
# 下载下载CodeFormer的权重
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer
# 安装CodeFormer的依赖
%cd /content/Double_Talker/CodeFormer
!python basicsr/setup.py develop

/content/Double_Talker
Cloning into 'CodeFormer'...
remote: Enumerating objects: 594, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 594 (delta 175), reused 170 (delta 148), pack-reused 349
Receiving objects: 100% (594/594), 17.31 MiB | 36.03 MiB/s, done.
Resolving deltas: 100% (286/286), done.
/content/Double_Talker/CodeFormer
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/Double_Talker/CodeFormer/weights/facelib/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 151MB/s]
Download

In [3]:
#@title 三、上传视频
%cd /content/Double_Talker/temp_video
!rm README.md
from google.colab import files
from io import BytesIO
uploaded = files.upload()
# 可以上传自己的视频，也可以用我的视频进行演示，点击取消上传即可

/content/Double_Talker/temp_video


In [4]:
#@title 四、上传音频
%cd /content/Double_Talker/input_audio
!rm README.md
from google.colab import files
from io import BytesIO
uploaded = files.upload()
# 可以上传自己的音频，也可以用我的音频进行演示，点击取消上传即可

/content/Double_Talker/input_audio


In [5]:
#@title 五、双人数字人视频预处理
%cd /content/Double_Talker
!mkdir input_video
!mkdir ls_video
!mkdir output
!python video_preprocessing.py

/content/Double_Talker
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag

In [6]:
#@title 六、界面化处理
# 使用"批处理" 输入 /content/Double_Talker/input_video  和 /content/Double_Talker/input_audio
%cd /content/Double_Talker/Wav2Lip
# 将wav2lip_gradio.py移动至Wav2lip文件下
!mv /content/Double_Talker/wav2lip_gradio.py  /content/Double_Talker/Wav2Lip
!python wav2lip_gradio.py

# 运行完后，请点击运行关闭

/content/Double_Talker/Wav2Lip
/content/Double_Talker/Wav2Lip/wav2lip_gradio.py:63: GradioUnusedKwargWarning: You have unused kwarg parameters in Video, please remove them: {'type': 'filepath'}
  video_input=gr.Video(label="输入视频(.mp4)", type="filepath", format="mp4")
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://71de7b65cfc7dfead3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 250
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping -

In [7]:
#@title 七、将单人数字人合并成双人数字人
%cd /content/Double_Talker
!python merge_video.py

/content/Double_Talker
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis

In [8]:
#@title 八、将生成的数字人视频连接起来
# 到这一步双人数字人已经运行完毕，后面是进行超分处理。
%cd /content/Double_Talker
# !ffmpeg -i "concat:/content/Double_Talker/output/ouput_s1.mp4|/content/Double_Talker/output/ouput_s2.mp4" -codec copy /content/Double_Talker/final_output/final.mp4
# 将视频连接起来
!ffmpeg -f concat -safe 0 -i <(for f in output/*.mp4; do echo "file '$PWD/$f'"; done) -c copy /content/Double_Talker/final_output/final.mp4
# 提取视频中的音频，用于后续处理
!ffmpeg -i /content/Double_Talker/final_output/final.mp4 /content/Double_Talker/final_output/final.mp3

/content/Double_Talker
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis

In [9]:
#@title 九、将推理出来的双人 数字人视频进行超分。
# 将Wav2Lip生成的视频转化成图片保存到images，用于后期高清化处理
%cd /content/Double_Talker/CodeFormer
!rm -rf images
!mkdir images
!ffmpeg -i /content/Double_Talker/final_output/final.mp4  -r 30  /content/Double_Talker/CodeFormer/images/%5d.png

/content/Double_Talker/CodeFormer
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enabl

In [ ]:
#@title 十、用CodeFormer进行高清化处理
%cd /content/Double_Talker/CodeFormer
!rm -rf results
!mkdir results
# 用CodeFormer进行推理
## 只修复人脸
# CODEFORMER_FIDELITY = 0.5
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/Double_Talker/CodeFormer
# whole images ##整体图片修复
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/Double_Talker/CodeFormer/images --bg_upsampler realesrgan

/content/Double_Talker/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/Double_Talker/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:03<00:00, 17.5MB/s]
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/1812] Processing: 00001.png
	detect 2 faces
[2/1812] Processing: 00002.png
	detect 2 faces
[3/1812] Processing: 00003.png
	detect 2 faces
[4/1812] Processing: 00004.png
	detect 2 faces
[5/1812] Processing: 00005.png
	detect 2 faces
[6/1812] Processing: 00006.png
	detect 2 faces
[7/1812] Processing: 00007.png
	detect 2 faces
[8/1812] Processing: 00008.png
	detect 2 faces
[9/1812] Processing: 00009.png
	detect 2 faces
[10/1812] Processing: 00010.png
	detect 2 faces
[11/1812] Processing: 00011.png
	detect 2 faces
[12/1812] Processing: 00012.png
	detect 2 faces
[13/1812] Processing: 00013.png
	detect 2 faces
[14/1812] Processing: 00014.png
	detec

In [ ]:
#@title 十一、合并成高清视频
# ffmpeg 将文件中每一帧图片合并成mp4
%cd /content/Double_Talker/CodeFormer/results/images_0.7/final_results
!ffmpeg -framerate 30 -i %05d.png  -pix_fmt yuv420p  /content/Double_Talker/CodeFormer/results/images_0.7/high_definition.mp4
%cd /content/
# 音视频合成
!ffmpeg -i /content/Double_Talker/CodeFormer/results/images_0.7/high_definition.mp4 -i /content/Double_Talker/final_output/final.mp3 -c copy /content/Double_Talker/final_output/high_definition_final.mp4
print('恭喜您高清数字人视频合并成功,')

[Errno 2] No such file or directory: '/content/CodeFormer/results/images_0.7/final_results'
/content/CodeFormer
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --e